In [115]:
import pandas as pd,numpy as np, seaborn as sns, matplotlib.pyplot as plt, joblib
from imblearn.over_sampling import SMOTE
from scipy import stats
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score

In [116]:
df = pd.read_csv('../0. Dataset/Training set after splitting.csv')
df.head()

,CustomerID,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,...,IncomeGroup,OwnsMotorcycle,AdjustmentsToCreditRating,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus,Churn
0,3317770,39.79,234.00,55.00,0.25,0.00,1.70,31.00,8.80,6.70,...,0,No,0,Unknown,No,2-High,Suburban,Other,Unknown,No
1,3224894,60.19,551.00,60.00,0.74,0.00,0.30,-88.00,-10.20,2.70,...,7,No,0,150,No,2-High,Suburban,Crafts,Yes,No
2,3231266,85.41,642.00,28.00,0.25,134.00,0.00,129.00,22.80,29.70,...,4,No,0,Unknown,No,2-High,Suburban,Crafts,No,No
3,3184922,30.18,281.00,30.00,0.00,0.00,0.00,132.00,-0.20,4.00,...,3,No,0,150,No,3-Good,Suburban,Other,Unknown,Yes
4,3350054,179.29,1459.00,95.00,0.50,118.00,48.60,-1360.00,-77.00,21.70,...,4,No,0,Unknown,No,3-Good,Rural,Other,Unknown,Yes


In [117]:
df.duplicated().sum()

0

In [118]:
df.columns

Index(['CustomerID', 'MonthlyRevenue', 'MonthlyMinutes',
       'TotalRecurringCharge', 'DirectorAssistedCalls', 'OverageMinutes',
       'RoamingCalls', 'PercChangeMinutes', 'PercChangeRevenues',
       'DroppedCalls', 'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls',
       'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls', 'InboundCalls',
       'PeakCallsInOut', 'OffPeakCallsInOut', 'DroppedBlockedCalls',
       'CallForwardingCalls', 'CallWaitingCalls', 'MonthsInService',
       'UniqueSubs', 'ActiveSubs', 'ServiceArea', 'Handsets', 'HandsetModels',
       'CurrentEquipmentDays', 'AgeHH1', 'AgeHH2', 'ChildrenInHH',
       'HandsetRefurbished', 'HandsetWebCapable', 'TruckOwner', 'RVOwner',
       'Homeownership', 'BuysViaMailOrder', 'RespondsToMailOffers',
       'OptOutMailings', 'NonUSTravel', 'OwnsComputer', 'HasCreditCard',
       'RetentionCalls', 'RetentionOffersAccepted', 'NewCellphoneUser',
       'NotNewCellphoneUser', 'ReferralsMadeBySubscriber', 'IncomeGroup',
    

In [119]:
df.dtypes

CustomerID                     int64
MonthlyRevenue               float64
MonthlyMinutes               float64
TotalRecurringCharge         float64
DirectorAssistedCalls        float64
OverageMinutes               float64
RoamingCalls                 float64
PercChangeMinutes            float64
PercChangeRevenues           float64
DroppedCalls                 float64
BlockedCalls                 float64
UnansweredCalls              float64
CustomerCareCalls            float64
ThreewayCalls                float64
ReceivedCalls                float64
OutboundCalls                float64
InboundCalls                 float64
PeakCallsInOut               float64
OffPeakCallsInOut            float64
DroppedBlockedCalls          float64
CallForwardingCalls          float64
CallWaitingCalls             float64
MonthsInService                int64
UniqueSubs                     int64
ActiveSubs                     int64
ServiceArea                   object
Handsets                     float64
H

In [120]:
df.isna().sum()

CustomerID                     0
MonthlyRevenue               120
MonthlyMinutes               120
TotalRecurringCharge         120
DirectorAssistedCalls        120
OverageMinutes               120
RoamingCalls                 120
PercChangeMinutes            290
PercChangeRevenues           290
DroppedCalls                   0
BlockedCalls                   0
UnansweredCalls                0
CustomerCareCalls              0
ThreewayCalls                  0
ReceivedCalls                  0
OutboundCalls                  0
InboundCalls                   0
PeakCallsInOut                 0
OffPeakCallsInOut              0
DroppedBlockedCalls            0
CallForwardingCalls            0
CallWaitingCalls               0
MonthsInService                0
UniqueSubs                     0
ActiveSubs                     0
ServiceArea                   21
Handsets                       1
HandsetModels                  1
CurrentEquipmentDays           1
AgeHH1                       716
AgeHH2    

In [121]:
state_map = {
        'NYC': 'New York', 'LAX': 'California', 'CHI': 'Illinois', 'HAR': 'Connecticut',
        'DAL': 'Texas', 'SAN': 'Texas', 'MIA': 'Florida', 'MIL': 'Wisconsin', 'NCR': 'North Carolina',
        'PHI': 'Pennsylvania', 'OKC': 'Oklahoma', 'MIN': 'Minnesota', 'OHI': 'Ohio', 'IND': 'Indiana',
        'SFR': 'California', 'LOU': 'Kentucky', 'APC': 'Maryland', 'BOS': 'Massachusetts', 'NOL': 'Louisiana',
        'SEA': 'Washington', 'DET': 'Michigan', 'PHX': 'Arizona', 'NEV': 'Nevada', 'OMA': 'Nebraska',
        'NNY': 'New York', 'STL': 'Missouri', 'FLN': 'Florida', 'HOU': 'Texas', 'PIT': 'Pennsylvania',
        'KCY': 'Kansas', 'ATL': 'Georgia', 'AIR': 'South Carolina', 'NMX': 'New Mexico', 'SHE': 'Virginia',
        'LAU': 'Mississippi', 'SLC': 'Utah', 'HWI': 'Hawaii', 'NSH': 'Tennessee', 'BIR': 'Alabama',
        'AWI': 'Wisconsin', 'DEN': 'Colorado', 'NOR': 'Minnesota', 'GCW': 'Mississippi', 'NVU': 'Nevada',
        'LAW': 'Texas', 'VAH': 'Virginia', 'SDA': 'South Dakota', 'SEW': 'Oregon', 'HOP': 'Montana',
        'SFU': 'California'
    }

def get_state(code):
    if not isinstance(code, str) or len(code) < 3:
        return np.nan
    metro_code = code[:3] 
    if metro_code == 'SAN':
        area_code = code[-3:] if len(code) >= 9 else ''
        if area_code in ['512', '210', '830', '254', '361', '956']:  # Texas area codes
            return 'Texas'
        return 'California'  # Default for 'SAN' in other contexts 
    return state_map.get(metro_code, np.nan)

def selectBestFeatures(df, target):
    selected_features = []
    for column in df.columns:
        if column != target:
            corr,pval = stats.pearsonr(df[column],df[target])
            if pval < 0.05:
                selected_features.append(column)
            #print("'{column}' has correlation of '{corr}' and pvalue of '{pval}'".format(column=column,corr=corr,pval=pval))
    return selected_features

def remove_outliers_using_IQR(df,column):
    first_quartile, third_quartile = np.percentile(df[column],25), np.percentile(df[column],75)
    IQR = third_quartile - first_quartile
    floor = first_quartile - 1.5 * IQR
    cieling = third_quartile + 1.5 * IQR
    outlier_mask = (df[column] < floor) | (df[column] > cieling)
    print()
    df = df[~outlier_mask]
    return df

In [122]:
df.drop(columns=['CustomerID'],axis=1,inplace=True)
df['HandsetPrice'] = df['HandsetPrice'].replace('Unknown', np.nan)
df['MaritalStatus'] = df['MaritalStatus'].replace('Unknown', np.nan)

yes_no_features = ['Churn','ChildrenInHH','HandsetRefurbished','HandsetWebCapable','TruckOwner','RVOwner','BuysViaMailOrder',
     'RespondsToMailOffers','OptOutMailings','NonUSTravel','OwnsComputer','HasCreditCard','NewCellphoneUser','NotNewCellphoneUser',
     'OwnsMotorcycle','MadeCallToRetentionTeam']

keep_values = {'CreditRating': ['2-High','1-Highest','3-Good'], 'Occupation': ['Professional','Crafts'], 
               'PrizmCode': ['Suburban','Town'],'ServiceArea': ['Texas', 'New York', 'California', 'Florida',
               'Maryland', 'Illinois','Georgia', 'Ohio', 'North Carolina', 'Nevada', 'Pennsylvania','Michigan', 
               'Massachusetts' ,'Washington', 'Kansas'] }

for feature in yes_no_features:
    df[feature] = [1 if value == 'Yes' else 0 for value in df[feature]]
df['Homeownership'] = [1 if value == 'Known' else 0 for value in df['Homeownership']]
df['ServiceArea'] = df['ServiceArea'].apply(lambda code: get_state(code))
for feature, values in keep_values.items():
    df[feature] = [value if value in values else 'Other' for value in df[feature]]
df = pd.get_dummies(df,columns=['CreditRating','PrizmCode','Occupation','ServiceArea'])
df['MaritalStatus'] = [1 if status == 'Yes' else 0 if status =='No' else np.nan for status in df['MaritalStatus']]

In [123]:
df.head()

,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,BlockedCalls,...,ServiceArea_Massachusetts,ServiceArea_Michigan,ServiceArea_Nevada,ServiceArea_New York,ServiceArea_North Carolina,ServiceArea_Ohio,ServiceArea_Other,ServiceArea_Pennsylvania,ServiceArea_Texas,ServiceArea_Washington
0,39.79,234.00,55.00,0.25,0.00,1.70,31.00,8.80,6.70,0.70,...,0,0,0,0,0,0,0,0,0,0
1,60.19,551.00,60.00,0.74,0.00,0.30,-88.00,-10.20,2.70,32.70,...,0,0,0,0,0,0,0,0,0,0
2,85.41,642.00,28.00,0.25,134.00,0.00,129.00,22.80,29.70,0.00,...,0,0,0,1,0,0,0,0,0,0
3,30.18,281.00,30.00,0.00,0.00,0.00,132.00,-0.20,4.00,0.00,...,0,0,0,1,0,0,0,0,0,0
4,179.29,1459.00,95.00,0.50,118.00,48.60,-1360.00,-77.00,21.70,11.00,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
imputer = KNNImputer(weights = 'distance')
imputer.fit(df.drop(columns=['Churn'],axis=1))
joblib.dump(imputer,'NullImputer.h5')

In [124]:
imputer = joblib.load('NullImputer.h5')
imputed_df = imputer.transform(df.drop(columns=['Churn'],axis=1))
imputed_df = pd.DataFrame(imputed_df, columns= df.drop(columns=['Churn']).columns)
imputed_df['Churn'] = df['Churn']

In [125]:
selected_features = selectBestFeatures(imputed_df, 'Churn')

In [126]:
len(selected_features)

51

In [129]:
selected_features

['MonthlyRevenue',
 'MonthlyMinutes',
 'TotalRecurringCharge',
 'DirectorAssistedCalls',
 'OverageMinutes',
 'RoamingCalls',
 'PercChangeMinutes',
 'PercChangeRevenues',
 'DroppedCalls',
 'UnansweredCalls',
 'CustomerCareCalls',
 'ThreewayCalls',
 'ReceivedCalls',
 'OutboundCalls',
 'InboundCalls',
 'PeakCallsInOut',
 'OffPeakCallsInOut',
 'CallWaitingCalls',
 'MonthsInService',
 'UniqueSubs',
 'ActiveSubs',
 'Handsets',
 'HandsetModels',
 'CurrentEquipmentDays',
 'AgeHH1',
 'AgeHH2',
 'HandsetRefurbished',
 'HandsetWebCapable',
 'Homeownership',
 'BuysViaMailOrder',
 'RespondsToMailOffers',
 'RetentionCalls',
 'RetentionOffersAccepted',
 'NotNewCellphoneUser',
 'IncomeGroup',
 'AdjustmentsToCreditRating',
 'HandsetPrice',
 'MadeCallToRetentionTeam',
 'CreditRating_1-Highest',
 'CreditRating_2-High',
 'CreditRating_3-Good',
 'CreditRating_Other',
 'PrizmCode_Suburban',
 'PrizmCode_Town',
 'ServiceArea_Florida',
 'ServiceArea_Kansas',
 'ServiceArea_Maryland',
 'ServiceArea_Michigan',
 '

In [130]:
imputed_df = imputed_df[selected_features]
imputed_df['Churn'] = df['Churn']
imputed_df.to_csv('Processed data.csv',index=False)